In [1]:
import datetime, time, random, string

def one_station(name):
    # temp pattern
    month_avg = [27,31,44,58,70,79,83,81,74,61,46,32]
    shift = (random.random()-0.5) * 30
    month_avg = [m + shift + (random.random()-0.5) * 5 for m in month_avg]
    
    # rain pattern
    start_rain = [0.1,0.1,0.3,0.5,0.4,0.2,0.2,0.1,0.2,0.2,0.2,0.1]
    shift = (random.random()-0.5) * 0.1
    start_rain = [r + shift + (random.random() - 0.5) * 0.2 for r in start_rain]
    stop_rain = 0.2 + random.random() * 0.2

    # day's state
    today = datetime.date(2000, 1, 1)
    temp = month_avg[0]
    raining = False
    
    # gen weather
    while True:
        # choose temp+rain
        month = today.month - 1
        temp = temp * 0.8 + month_avg[month] * 0.2 + (random.random()-0.5) * 20
        if temp < 32:
            raining=False
        elif raining and random.random() < stop_rain:
            raining = False
        elif not raining and random.random() < start_rain[month]:
            raining = True

        yield (today.strftime("%Y-%m-%d"), name, temp, raining)

        # next day
        today += datetime.timedelta(days=1)
        
def all_stations(count=10, sleep_sec=1):
    assert count <= 26
    stations = []
    for name in string.ascii_uppercase[:count]:
        stations.append(one_station(name))
    while True:
        for station in stations:
            yield next(station)
        time.sleep(sleep_sec)

In [3]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer, TopicPartition
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError, UnknownTopicOrPartitionError

broker = "kafka:9092"
admin = KafkaAdminClient(bootstrap_servers =[broker])

In [4]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer, TopicPartition
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError, UnknownTopicOrPartitionError

broker = "kafka:9092"
admin = KafkaAdminClient(bootstrap_servers =[broker])
try:
    admin.delete_topics(["stations", "stations-json"])
    print("deleted")
except UnknownTopicOrPartitionError:
    print("cannot delete (may not exist yet)")

time.sleep(1)
admin.create_topics([NewTopic("stations", 6, 1)])
admin.create_topics([NewTopic("stations-json", 6, 1)])
admin.list_topics()

cannot delete (may not exist yet)


['stations-json', 'stations']

In [16]:
from message_pb2 import *
from threading import Thread
import threading,json
def produce():
    producer = KafkaProducer(bootstrap_servers=["kafka:9092"],retries=10,acks=-1)
    
    for date, station, degrees, raining in all_stations(15):
        #print(date, station, degrees, raining)
        s = Report(date=date, station=station,degrees=(degrees),raining=raining)
        value = s.SerializeToString()
        key=station.encode('utf-8')
        producer.send('stations',key=key ,value=value)
        value_json = {"date": date, "station": station,'degrees':(degrees),'raining':int(raining)}
        value_json = bytes(json.dumps(value_json), "utf-8")
        producer.send('stations-json',key=key,value=value_json)
        # TODO: send to "stations" stream using protobuf
        # TODO: send to "stations-json" using JSON

# TODO: start thread to run produce
threading.Thread(target=produce).start()
# never join thread because we want it to run forever

In [20]:
int(False)

0

In [23]:
value_json = {"beach": '2000-01-01', "animal": '343','degrees':int(45),'raining':int(False)}

In [24]:
value_json

{'beach': '2000-01-01', 'animal': '343', 'degrees': 45, 'raining': 0}